In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import nengo
import random
import pickle
import os

import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score
from sklearn.linear_model import LogisticRegression

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# SKLearn Logistic Regression

## Raw Data without LMU Preprocessing as Input

In [ ]:
def simpleLR_raw(run):
    
    data_dir = "./LR_Raw/"
    if run == 0:
        os.mkdir(data_dir)
    
    highdata_raw = np.load('higheng.npy',allow_pickle=True)
    lowdata_raw = np.load('loweng.npy',allow_pickle=True)
    middata_raw = np.load('mideng.npy',allow_pickle=True)
    randdata_raw=np.load('rand_data.npy',allow_pickle=True)

    high=list(highdata_raw[:,1])
    low=list(lowdata_raw[:,1])
    mid_test=list(middata_raw[:,1])
    rand_data=list(randdata_raw)

    #shuffle clips
    np.random.shuffle(high)
    np.random.shuffle(low)
    np.random.shuffle(mid_test)
    np.random.shuffle(rand_data)

    #get training clips and convert to list of frames
    high_train = high[:(int(len(low)*0.7))]
    high_train=list(np.vstack(high_train))
    low_train = low[:(int(len(low)*0.7))]
    low_train=list(np.vstack(low_train))

    #shuffle frames
    np.random.shuffle(high_train)
    np.random.shuffle(low_train)

    #Make each training set same size (noplay is shortest)
    high_train = high_train[:(int(len(low_train)))]
    low_train = low_train[:(int(len(low_train)))]

    #Create test sets
    high_test = high[(int(len(low)*0.7)):(int(len(low)*0.9))]
    low_test = low[(int(len(low)*0.7)):(int(len(low)*0.9))]
    rand_test = rand_data[:18]

    np.random.shuffle(high_test)
    np.random.shuffle(low_test)
    np.random.shuffle(mid_test)
    np.random.shuffle(rand_test)

    all_train = np.vstack(np.concatenate((high_train, low_train)))
    all_test = np.concatenate((high_test, mid_test, low_test, rand_test)) 

    pickle_filename = (data_dir+"/%s_train_data.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(all_train, file)
        
    pickle_filename = (data_dir+"/%s_test_data.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(all_test, file)

    #create the target data for training
    target_train = np.zeros((all_train.shape[0],1))
    n_high = len(high_train)
    target_train[:n_high,0] = 1
    target_train[n_high:,0] = -1

    #and for testing
    target_test = np.zeros((all_test.shape[0],1))
    n_high = len(high_test)
    n_mid = n_high + len(mid_test)
    n_low = n_mid + len(low_test)
    target_test[:n_high,0] = 1
    target_test[n_high:n_mid,0] = 0
    target_test[n_mid:n_low,0] = -1
    target_test[n_low:,0] = 0 #np.random.rand()
    
    classifier = LogisticRegression(max_iter=1000)
    classifier.fit(all_train, target_train)

    y_pred=[]
    decision=[]
    prob=[]
    for i in all_test:
        y_pred.append(classifier.predict(i))
        decision.append(classifier.decision_function(i))
        prob.append(classifier.predict_proba(i))

    pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(y_pred, file)
        
    pickle_filename = (data_dir+"/%s_decision.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(decision, file)
        
    pickle_filename = (data_dir+"/%s_prob.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(prob, file)
        
    pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(target_test, file)
    
    return classifier

In [ ]:
for run in range(20):
    classifier = simpleLR_raw(run)

## LMU Preprocessed Data as Input

In [ ]:
def simpleLR_lmu(run):
    
    data_dir = "./LR_LMU/"
    if run == 0:
        os.mkdir(data_dir)
    
    highdata_lmu = np.load('high_lmu4_theta3.npy',allow_pickle=True)
    lowdata_lmu = np.load('low_lmu4_theta3.npy',allow_pickle=True)
    middata_lmu = np.load('mid_lmu4_theta3.npy',allow_pickle=True)
    rand_lmu = np.load('rand_lmu4_theta3.npy', allow_pickle=True)
    
    high=list(highdata_lmu[:])
    low=list(lowdata_lmu[:])
    mid_test=list(middata_lmu[:])
    rand_data=list(rand_lmu)

    #shuffle clips
    np.random.shuffle(high)
    np.random.shuffle(low)
    np.random.shuffle(mid_test)
    np.random.shuffle(rand_data)

    #get training clips and convert to list of frames
    high_train = high[:(int(len(low)*0.7))]
    high_train=list(np.vstack(high_train))
    low_train = low[:(int(len(low)*0.7))]
    low_train=list(np.vstack(low_train))

    #shuffle frames
    np.random.shuffle(high_train)
    np.random.shuffle(low_train)

    #Make each training set same size (noplay is shortest)
    high_train = high_train[:(int(len(low_train)))]
    low_train = low_train[:(int(len(low_train)))]

    #Create test sets
    high_test = high[(int(len(low)*0.7)):(int(len(low)*0.9))]
    low_test = low[(int(len(low)*0.7)):(int(len(low)*0.9))]
    rand_test = rand_data[:18]

    np.random.shuffle(high_test)
    np.random.shuffle(low_test)
    np.random.shuffle(mid_test)
    np.random.shuffle(rand_test)

    all_train = np.vstack(np.concatenate((high_train, low_train)))
    all_test = np.concatenate((high_test, mid_test, low_test, rand_test))
    
    pickle_filename = (data_dir+"/%s_train_data.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(all_train, file)
        
    pickle_filename = (data_dir+"/%s_test_data.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(all_test, file)

    #create the target data for training
    target_train = np.zeros((all_train.shape[0],1))
    n_high = len(high_train)
    target_train[:n_high,0] = 1
    target_train[n_high:,0] = -1

    #and for testing
    target_test = np.zeros((all_test.shape[0],1))
    n_high = len(high_test)
    n_mid = n_high + len(mid_test)
    n_low = n_mid + len(low_test)
    target_test[:n_high,0] = 1
    target_test[n_high:n_mid,0] = 0
    target_test[n_mid:n_low,0] = -1
    target_test[n_low:,0] = 0 #np.random.rand()   
    
    classifier = LogisticRegression(max_iter=1000)
    classifier.fit(all_train, target_train)

    y_pred=[]
    decision=[]
    prob=[]
    accuracy=[]
    for i in all_test:
        y_pred.append(classifier.predict(i))
        decision.append(classifier.decision_function(i))
        prob.append(classifier.predict_proba(i))

    pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(y_pred, file)
        
    pickle_filename = (data_dir+"/%s_decision.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(decision, file)
        
    pickle_filename = (data_dir+"/%s_prob.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(prob, file)
        
    pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run)
    with open(pickle_filename, 'wb') as file:
        pickle.dump(target_test, file)
    
    return classifier

In [ ]:
for run in range(20):
    classifier = simpleLR_lmu(run)